In [1]:
import numpy
from etrade.spot.trader import Station
from etrade.spot.market.recycle import BasicRecycle, PointwiseRecycle
from etrade.spot.forecast.market import DistributiveSeries, DistributiveMarket
from etrade.spot.forecast.plan_a.constructor import *
from  data_utils.stochastic_utils.vdistributions.abstract import AbstractDistribution
from  data_utils.stochastic_utils.vdistributions.parameter.continuous.basic import NormalDistribution, SkewNormalDistribution
from data_utils.stochastic_utils.vdistributions.tools.divergence import kl_divergence_continuous, crps

from data_utils.stochastic_utils.vdistributions.nonparametric.continuous.mfk.skewnd2 import dof_snd_fitter

from scipy.optimize import differential_evolution

from matplotlib import pyplot

In [2]:
aq = OrdinaryGaussianKernelDistributionConstructor((20, 50), (1, 10), (2, 4))
dp = OrdinaryGaussianKernelDistributionConstructor((0, 5), (1, 5), (2, 4))
rp = OrdinaryGaussianKernelDistributionConstructor((0, 5), (1, 5), (2, 4))

In [3]:
mc = MarketConstructor(aq, dp, rp)
s = Station("station", 50)
br = PointwiseRecycle()

In [4]:
real_market_distribution = mc.clamped_random(4, (0, 50), (0, numpy.inf), (0, numpy.inf))
predicted_market_distribtion = mc.clamped_random(4, (0, 50), (0, numpy.inf), (0, numpy.inf))

In [5]:
observed_data = numpy.asarray(real_market_distribution.rvf(1)).reshape(3,-1)

In [6]:
predicted_market_distribtion.crps(observed_data[0], observed_data[1], observed_data[2])

array([[15.06279584,  3.56956953,  6.95783733, 10.3838931 ],
       [ 1.3144361 ,  1.19461242,  5.45897925,  1.19499619],
       [ 1.25281799,  1.69275024,  1.15751538,  1.0972816 ]])

In [7]:
predicted_market_distribtion.price_kl_divergence()

array([ 0.26636479, -0.02201267, -0.01653685,  0.26502363])

In [8]:
predicted_market_distribtion.quantity_optimizer(s, br).x

array([33.3333855 , 29.214725  , 33.42837457, 33.51642006])

In [9]:
predicted_market_distribtion.faster_quantity_optimizer(s, br).x

array([42.14518825, 28.61636362, 32.25386912, 37.59427104])

In [10]:
predicted_market_distribtion.rvf(10)[0][:,0]

array([50.00007204, 41.44999459, 48.22069536, 28.82387541])